In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import PchipInterpolator
from ipywidgets import interact, interactive, FloatSlider, IntSlider, Dropdown
from scipy.special import expita


In [2]:
# t0 - duration of phase 0
# t1 - duration of phase 1
# t2 - duration of phase 2
# t3 - duration of phase 3
# For atrial AP, duration of phase 4 is not relevant, because it is just constant potential until disturbed by the signals from other cells

# t01 - time of shift from phase 0 to phase 1
# t12 - time of shift from phase 1 to phase 2
# t23 - time of shift from phase 2 to phase 3
# t34 - time of shift from phase 3 to phase 4

# V01 - potential at t01 (peak potential)
# V12 - potential at t12 (plateau potential)
# V23 - potential at t23 (late repolarization potential)
# V34 - potential at t34 (true resting membrane potential)

# translation from variables to sliders:

# V01 - Peak Membrane Potential
# V12 - Plateau Potential
# V23 - Late Repolarization Potential
# V34 - Resting Membrane Potential

# t0 - Rapid Depolarization Duration
# t1 - Early Rapid Repolarization Duration
# t2 - Plateau Duration
# t3 - Final Repolarizarion Duration


In [3]:
def fast_upstroke(t, V_start, V_end, t01):
    """Steep sigmoid upstroke, numerically stable"""
    k = max(t01, 1e-4) / 12.0
    t_mid = 0.6 * t01
    return V_start + (V_end - V_start) * expit((t - t_mid)/k)

In [4]:
def atrial_AP(t,
              V34=-85.0,
              V01=+20.0,
              V12=+5.0,
              V23=-15.0,
              t01=0.004,
              t12=0.015,
              t23=0.070,
              t34=0.170,
              t40=2.0):

    # Phase 4 duration
    t_rest = t40 - t34
    t_shift = t - t_rest

    # Phase 0
    P0 = fast_upstroke(t_shift, V34, V01, t01)

    # Phases 1-3
    knots_t = [t01, t12, t23, t34]
    knots_V = [V01, V12, V23, V34]
    spline = PchipInterpolator(knots_t, knots_V)
    P13 = spline(t_shift)

    # Total potential
    V = np.where(t < t_rest, V34,
                 np.where(t_shift < t01, P0, P13))
    return V

In [5]:
def interactive_atrial(V01=+20.0, 
                       V12=+5.0, 
                       V23=-15.0,
                       V34=-85.0, 
                       t0=4, 
                       t1=15, 
                       t2=70,
                       t3=900, 
                       t4=1.80):
    
    t0, t1, t2, t3, t4 = (x / 1000 for x in (t0, t1, t2, t3, t4))    
    
    t01 = t0
    t12 = t01 + t1
    t23 = t12 + t2
    t34 = t23 + t3
    t40 = t34 + t4

    t = np.linspace(0, t40, 800)
    V = atrial_AP(t, 
                  V34, V01, V12, V23, 
                  t01, t12, t23, t34, t40)

    plt.figure(figsize=(10,4))
    plt.plot(t*1000, V, lw=2)
    plt.xlabel("Time (ms)")
    plt.ylabel("Voltage (mV)")
    plt.title(f"Simulated Atrial Action Potential")
    plt.grid(True)
    plt.ylim(V34-10, V01+20)
    plt.show()

In [6]:
slider_layout = {'width': '600px'} 
slider_style = {'description_width': '250px'}

In [ ]:
interact(interactive_atrial,
         V01=FloatSlider(value=20, min=10, max=40, step=1,
                         description="Peak Membrane Potential (mV)",
                         style=slider_style, layout=slider_layout),
         V12=FloatSlider(value=5, min=-20, max=20, step=1,
                         description="Plateau Potential (mV)",
                         style=slider_style, layout=slider_layout),
         V23=FloatSlider(value=-15, min=-40, max=0, step=1,
                         description="Late Repolarization Potential (mV)",
                         style=slider_style, layout=slider_layout),
        V34=FloatSlider(value=-85, min=-100, max=-75, step=1,
                        description="Resting Membrane Potential (mV)",
                        style=slider_style, layout=slider_layout),
         t0=FloatSlider(value=4, min=1, max=4, step=10,
                        description="Rapid Depolarization Duration (ms)",
                        style=slider_style, layout=slider_layout),
         t1=FloatSlider(value=4, min=2, max=6, step=1,
                        description="Early Rapid Repolarization Duration (ms)",
                        style=slider_style, layout=slider_layout),
         t2=FloatSlider(value=150, min=120, max=180, step=10,
                        description="Plateau Duration (ms)",
                        style=slider_style, layout=slider_layout),
         t3=FloatSlider(value=80, min=50, max=100, step=10,
                        description="Final Repolarizarion Duration (ms)",
                        style=slider_style, layout=slider_layout),
         t4=FloatSlider(value=30, min=10, max=200, step=10,
                        description="Resting State Duration (ms)",
                        style=slider_style, layout=slider_layout),
        );

interactive(children=(FloatSlider(value=20.0, description='Peak Membrane Potential (mV)', layout=Layout(width=…